
<p style="text-align: center;font-size: 18px;">
    BIO/MIC 494/598 <span style="font-weight: bold;">Microbiome Data Science</span>
</p>
<p style="text-align: center;"><span style="font-size: 32px; font-weight: bold; background-color: #ffc627;">
    Sequencing Data Processing
</span></p>
<p style="text-align: center; font-size: 18px;">
    Team 5
</p>

## Data files

In [22]:
! git clone https://github.com/salorain/10251.git   

Cloning into '10251'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.


In [31]:
!cd 10251

## From Qiita to Qiime2

In [33]:
!qiime tools import \
  --input-path 58889_reference-hit.biom \
  --type 'FeatureTable[Frequency]' \
  --output-path 10251table.qza


Imported 58889_reference-hit.biom as BIOMV210DirFmt to 10251table.qza


In [26]:
!qiime tools import \
  --input-path 58889_reference-hit.seqs.fa \
  --type 'FeatureData[Sequence]' \
  --output-path 10251rep-seqs.qza


Imported 58889_reference-hit.seqs.fa as DNASequencesDirectoryFormat to 10251rep-seqs.qza


In [27]:
!qiime tools import \
  --input-path 58889_insertion_tree.relabelled.tre \
  --type 'Phylogeny[Rooted]' \
  --output-path 10251rooted-tree.qza

Imported 58889_insertion_tree.relabelled.tre as NewickDirectoryFormat to 10251rooted-tree.qza


In [5]:
!qiime feature-table summarize \
  --i-table 10251table.qza \
  --o-visualization 10251table.qzv \
  --m-sample-metadata-file 10251_prep_690_20201201-080233.txt
!qiime feature-table tabulate-seqs \
  --i-data 10251rep-seqs.qza \
  --o-visualization 10251rep-seqs.qzv

Saved Visualization to: 10251table.qzv
Saved Visualization to: 10251rep-seqs.qzv


In [ ]:
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences 10251rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree 10251unrooted-tree.qza \
  --o-rooted-tree 102511rooted-tree.qza

## Taxonomic analysis

In [37]:
!curl -sL \
  "https://data.qiime2.org/2021.2/common/gg-13-8-99-515-806-nb-classifier.qza" > \
  "gg-13-8-99-515-806-nb-classifier.qza"

In [38]:
!qiime feature-classifier classify-sklearn \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads 10251rep-seqs.qza \
  --o-classification taxonomy.qza

!qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

Saved FeatureData[Taxonomy] to: taxonomy.qza
Saved Visualization to: taxonomy.qzv


# Rarefaction

In [1]:
!qiime feature-table rarefy \
  --i-table 10251table.qza \
  --p-sampling-depth 20102 \
  --o-rarefied-table rarefied_table.qza

Saved FeatureTable[Frequency] to: rarefied_table.qza


Retained 5,849,682 (26.52%) features in 291 (98.64%) samples at the specifed sampling depth (20102).

In [3]:
!qiime feature-table summarize \
  --i-table rarefied_table.qza \
  --o-visualization rarefied_table.qzv \
  --m-sample-metadata-file 10251/10251_prep_690_20201201-080233.txt

Saved Visualization to: rarefied_table.qzv


### Alpha and beta diversity analysis

In [5]:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny 102511rooted-tree.qza \
  --i-table 10251table.qza \
  --p-sampling-depth 20102 \
  --m-metadata-file 10251/10251_prep_690_20201201-080233.txt \
  --output-dir all_in_one

Saved FeatureTable[Frequency] to: all_in_one/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: all_in_one/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: all_in_one/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: all_in_one/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: all_in_one/evenness_vector.qza
Saved DistanceMatrix to: all_in_one/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: all_in_one/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: all_in_one/jaccard_distance_matrix.qza
Saved DistanceMatrix to: all_in_one/bray_curtis_distance_matrix.qza
Saved PCoAResults to: all_in_one/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: all_in_one/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: all_in_one/jaccard_pcoa_results.qza
Saved PCoAResults to: all_in_one/bray_curtis_pcoa_results.qza
Saved Visualization to: all_in_one/unweighted_unifrac_emperor.qzv
Saved Visualization to: all_in_one/weighted_unifrac_em

## Faith Phylogenetic Diversity

In [8]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file 10251_prep_690_20201201-080233.txt \
  --o-visualization core-metrics-results/faith-pd-group-significance.qzv

Saved Visualization to: core-metrics-results/faith-pd-group-significance.qzv


phylogeny-informed alpha diversity using the Faith's PD metric

In [8]:
!qiime diversity alpha-phylogenetic \
  --i-table rarefied_table.qza \
  --i-phylogeny 102511rooted-tree.qza \
  --p-metric faith_pd \
  --o-alpha-diversity faith_pd_vector.qza

Saved SampleData[AlphaDiversity] to: faith_pd_vector.qza


#### Evenness metrics

In [9]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/evenness_vector.qza \
  --m-metadata-file 10251_prep_690_20201201-080233.txt \
  --o-visualization core-metrics-results/evenness-group-significance.qzv

Saved Visualization to: core-metrics-results/evenness-group-significance.qzv


### Beta Diversity using unweighted UniFrac distances for expermint title (Utilities from A to E )

In [10]:
!qiime diversity beta-group-significance   \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza   \
  --m-metadata-file 10251_prep_690_20201201-080233.txt   --m-metadata-column experiment_title   \
  --o-visualization core-metrics-results/unweighted-unifrac-expermint-title-significance.qzv   \
  --p-pairwise

Saved Visualization to: core-metrics-results/unweighted-unifrac-expermint-title-significance.qzv


#### Alpha group significance

In [10]:
%%bash
for metric in observed_features shannon pielou_e faith_pd
do
  qiime diversity alpha-group-significance \
    --i-alpha-diversity ${metric}_vector.qza \
    --m-metadata-file 10251/10251_prep_690_20201201-080233.txt \
    --o-visualization ${metric}_group_significance.qzv
done

Saved Visualization to: observed_features_group_significance.qzv
Saved Visualization to: shannon_group_significance.qzv
Saved Visualization to: pielou_e_group_significance.qzv
Saved Visualization to: faith_pd_group_significance.qzv


In [6]:
%%bash
for metric in observed_features shannon pielou_e
do
  qiime diversity alpha \
    --i-table rarefied_table.qza \
    --p-metric $metric \
    --o-alpha-diversity ${metric}_vector.qza
done

Saved SampleData[AlphaDiversity] to: observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: shannon_vector.qza
Saved SampleData[AlphaDiversity] to: pielou_e_vector.qza


#### Alpha rarefaction plotting

In [6]:
!qiime diversity alpha-rarefaction \
  --i-table 10251table.qza \
  --i-phylogeny 102511rooted-tree.qza \
  --p-max-depth 100000 \
  --m-metadata-file 10251/10251_prep_690_20201201-080233.txt \
  --o-visualization alpha-rarefaction.qzv

Saved Visualization to: alpha-rarefaction.qzv


#### Beta Diversity

In [19]:
%%bash
for metric in jaccard braycurtis
do
  qiime diversity beta \
    --i-table rarefied_table.qza \
    --p-metric $metric \
    --o-distance-matrix ${metric}_distance_matrix.qza
done

Saved DistanceMatrix to: jaccard_distance_matrix.qza
Saved DistanceMatrix to: braycurtis_distance_matrix.qza


In [22]:
%%bash
for metric in unweighted_unifrac weighted_unifrac
do
  qiime diversity beta-phylogenetic \
    --i-table rarefied_table.qza \
    --i-phylogeny 102511rooted-tree.qza \
    --p-metric $metric \
    --o-distance-matrix ${metric}_distance_matrix.qza
done

Saved DistanceMatrix to: unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: weighted_unifrac_distance_matrix.qza


#### PCoA

In [23]:
%%bash
for metric in jaccard braycurtis unweighted_unifrac weighted_unifrac
do
  qiime diversity pcoa \
    --i-distance-matrix ${metric}_distance_matrix.qza \
    --o-pcoa ${metric}_pcoa_results.qza
done

Saved PCoAResults to: jaccard_pcoa_results.qza
Saved PCoAResults to: braycurtis_pcoa_results.qza
Saved PCoAResults to: unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: weighted_unifrac_pcoa_results.qza


#### EMPeror

In [25]:
%%bash
for metric in jaccard braycurtis unweighted_unifrac weighted_unifrac
do
  qiime emperor plot \
    --i-pcoa ${metric}_pcoa_results.qza \
    --m-metadata-file 10251/10251_prep_690_20201201-080233.txt \
    --o-visualization ${metric}_emperor.qzv
done

Saved Visualization to: jaccard_emperor.qzv
Saved Visualization to: braycurtis_emperor.qzv
Saved Visualization to: unweighted_unifrac_emperor.qzv
Saved Visualization to: weighted_unifrac_emperor.qzv


#### Time series

In [28]:
!qiime emperor plot \
  --i-pcoa unweighted_unifrac_pcoa_results.qza \
  --m-metadata-file 10251/10251_prep_690_20201201-080233.txt \
  --p-custom-axes run_date \
  --o-visualization unweighted_unifrac_emperor_time.qzv

Plugin error from emperor:

  All values in a custom axis must be numeric, this summary shows the invalid values and the number of times they were found in column 'run_date':
  8/6/13     178
  5/15/13    113

Debug info has been saved to /var/folders/nb/_drlqg395x792y1_46fbz9lm0000gn/T/qiime2-q2cli-err-o_f13dio.log


Did not work in our stuy case



#### Beta rarefaction


In [30]:
!qiime diversity beta-rarefaction \
  --i-table 10251table.qza \
  --i-phylogeny 102511rooted-tree.qza\
  --p-metric unweighted_unifrac \
  --p-clustering-method nj \
  --p-sampling-depth 1103 \
  --m-metadata-file 10251/10251_prep_690_20201201-080233.txt \
  --o-visualization unweighted_unifrac_rarefaction.qzv

Saved Visualization to: unweighted_unifrac_rarefaction.qzv


#### PCoA biplot

In [31]:
!qiime feature-table relative-frequency \
  --i-table rarefied_table.qza \
  --o-relative-frequency-table relative_table.qza

Saved FeatureTable[RelativeFrequency] to: relative_table.qza


In [32]:
!qiime diversity pcoa-biplot \
  --i-pcoa unweighted_unifrac_pcoa_results.qza \
  --i-features relative_table.qza \
  --o-biplot unweighted_unifrac_pcoa_biplot.qza

Saved PCoAResults % Properties('biplot') to: unweighted_unifrac_pcoa_biplot.qza


In [39]:
!qiime emperor biplot \
  --i-biplot unweighted_unifrac_pcoa_biplot.qza \
  --m-sample-metadata-file 10251/10251_prep_690_20201201-080233.txt \
  --m-feature-metadata-file taxonomy.qza \
  --o-visualization unweighted_unifrac_emperor_biplot.qzv

Saved Visualization to: unweighted_unifrac_emperor_biplot.qzv


#### Mantel test¶


In [40]:
!qiime diversity mantel \
  --i-dm1 braycurtis_distance_matrix.qza \
  --i-dm2 weighted_unifrac_distance_matrix.qza \
  --o-visualization bc_vs_wu_mantel.qzv

Saved Visualization to: bc_vs_wu_mantel.qzv


#### Procrustes analysis¶


In [41]:
!qiime diversity procrustes-analysis \
  --i-reference braycurtis_pcoa_results.qza \
  --i-other weighted_unifrac_pcoa_results.qza \
  --o-transformed-reference braycurtis_pcoa_transformed.qza \
  --o-transformed-other weighted_unifrac_pcoa_transformed.qza

Saved PCoAResults to: braycurtis_pcoa_transformed.qza
Saved PCoAResults to: weighted_unifrac_pcoa_transformed.qza


In [43]:
!qiime emperor procrustes-plot \
  --i-reference-pcoa braycurtis_pcoa_transformed.qza \
  --i-other-pcoa weighted_unifrac_pcoa_transformed.qza \
  --m-metadata-file 10251/10251_prep_690_20201201-080233.txt \
  --o-visualization bc_vs_wu_procrustes.qzv

Saved Visualization to: bc_vs_wu_procrustes.qzv


#### Group significance

In [2]:
%%bash
for test in anosim permanova permdisp
do
  qiime diversity beta-group-significance \
    --i-distance-matrix unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file 10251/10251_prep_690_20201201-080233.txt \
    --m-metadata-column "experiment_title" \
    --p-method $test \
    --o-visualization unweighted_unifrac_body_site_${test}.qzv
done

Saved Visualization to: unweighted_unifrac_body_site_anosim.qzv
Saved Visualization to: unweighted_unifrac_body_site_permanova.qzv
Saved Visualization to: unweighted_unifrac_body_site_permdisp.qzv


#### Adonis test

In [3]:
!cat 10251/10251_prep_690_20201201-080233.txt | tr -d '-' > 10251/10251_prep_690_20201201-080233_rev.txt


In [6]:
!qiime diversity adonis \
  --i-distance-matrix unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file 10251/10251_prep_690_20201201-080233_rev.txt \
  --p-formula "experiment_title" \
  --o-visualization unweighted_unifrac_body_site_adonis.qzv

Saved Visualization to: unweighted_unifrac_body_site_adonis.qzv


### Extracting data from QIIME 2 results

In [20]:
import pandas as pd

In [21]:
from qiime2 import Metadata


In [22]:
meta = Metadata.load('10251/10251_prep_690_20201201-080233.txt')


In [23]:
meta

Metadata
--------
295 IDs x 21 columns
barcode:                       ColumnProperties(type='categorical')
center_name:                   ColumnProperties(type='categorical')
center_project_name:           ColumnProperties(type='categorical')
experiment_center:             ColumnProperties(type='categorical')
experiment_design_description: ColumnProperties(type='categorical')
experiment_title:              ColumnProperties(type='categorical')
instrument_model:              ColumnProperties(type='categorical')
library_construction_protocol: ColumnProperties(type='categorical')
linker:                        ColumnProperties(type='categorical')
pcr_primers:                   ColumnProperties(type='categorical')
platform:                      ColumnProperties(type='categorical')
primer:                        ColumnProperties(type='categorical')
qiita_prep_id:                 ColumnProperties(type='numeric')
run_center:                    ColumnProperties(type='categorical')
run_date:    

## visualization

In [24]:
from qiime2 import Visualization


In [25]:
df = meta.to_dataframe()
df.head()

,barcode,center_name,center_project_name,experiment_center,experiment_design_description,experiment_title,instrument_model,library_construction_protocol,linker,pcr_primers,...,primer,qiita_prep_id,run_center,run_date,run_prefix,samp_size,sample_center,sequencing_meth,target_gene,target_subfragment
sample_name,,,,,,,,,,,,,,,,,,,,,
10251.A.B.1.1D,TCCCTTGCTTCCT,Virginia Tech,Field Water Utility,Virginia Tech,drinking water from standardized rigs,Utility A,Illumina MiSeq,"Illumina 16S V4 515f, 806rbc, EMP old protocol",GT,FWD:GTGCCAGCMGCCGCGGTAA; REV:GGACTACHVGGGTWTCTAAT,...,GTGCCAGCMGCCGCGGTAA,690.0,VBI,8/6/13,GRL2085_S34_L001_R1_001,".1,g",Virginia Tech,Sequencing by synthesis,16S rRNA,V4
10251.A.B.1.1W,TCCCTTGCTTCCT,Virginia Tech,Field Water Utility,Virginia Tech,drinking water from standardized rigs,Utility A,Illumina MiSeq,"Illumina 16S V4 515f, 806rbc, EMP old protocol",GT,FWD:GTGCCAGCMGCCGCGGTAA; REV:GGACTACHVGGGTWTCTAAT,...,GTGCCAGCMGCCGCGGTAA,690.0,VBI,8/6/13,GRL2055_S4_L001_R1_001,".1,g",Virginia Tech,Sequencing by synthesis,16S rRNA,V4
10251.A.B.1.2D,TCCCTTGCTTCCT,Virginia Tech,Field Water Utility,Virginia Tech,drinking water from standardized rigs,Utility A,Illumina MiSeq,"Illumina 16S V4 515f, 806rbc, EMP old protocol",GT,FWD:GTGCCAGCMGCCGCGGTAA; REV:GGACTACHVGGGTWTCTAAT,...,GTGCCAGCMGCCGCGGTAA,690.0,VBI,8/6/13,GRL2095_S44_L001_R1_001,".1,g",Virginia Tech,Sequencing by synthesis,16S rRNA,V4
10251.A.B.1.2W,TCCCTTGCTTCCT,Virginia Tech,Field Water Utility,Virginia Tech,drinking water from standardized rigs,Utility A,Illumina MiSeq,"Illumina 16S V4 515f, 806rbc, EMP old protocol",GT,FWD:GTGCCAGCMGCCGCGGTAA; REV:GGACTACHVGGGTWTCTAAT,...,GTGCCAGCMGCCGCGGTAA,690.0,VBI,8/6/13,GRL2065_S14_L001_R1_001,".1,g",Virginia Tech,Sequencing by synthesis,16S rRNA,V4
10251.A.B.1.3D,TCCCTTGCTTCCT,Virginia Tech,Field Water Utility,Virginia Tech,drinking water from standardized rigs,Utility A,Illumina MiSeq,"Illumina 16S V4 515f, 806rbc, EMP old protocol",GT,FWD:GTGCCAGCMGCCGCGGTAA; REV:GGACTACHVGGGTWTCTAAT,...,GTGCCAGCMGCCGCGGTAA,690.0,VBI,8/6/13,GRL2105_S54_L001_R1_001,".1,g",Virginia Tech,Sequencing by synthesis,16S rRNA,V4


In [81]:
table_qzv = Visualization.load('10251table.qzv')

#### Faith Phylogenetic Diversity

In [82]:
faith_pd_group_significance = Visualization.load('core-metrics-results/faith-pd-group-significance.qzv')

#### Rarefaction Visualization

In [83]:
rarefied_table = Visualization.load('rarefied_table.qzv')

#### Alpha and beta diversity analysis Visualization

In [84]:
unweighted_unifrac_emperor = Visualization.load('all_in_one/unweighted_unifrac_emperor.qzv')

In [85]:
weighted_unifrac_emperor = Visualization.load('all_in_one/weighted_unifrac_emperor.qzv')

In [86]:
jaccard_emperor = Visualization.load('all_in_one/jaccard_emperor.qzv')

In [87]:
bray_curtis_emperor = Visualization.load('all_in_one/bray_curtis_emperor.qzv')

#### Evenness metrics Visualization

In [88]:
evenness_group_significance = Visualization.load('core-metrics-results/evenness-group-significance.qzv')

#### Beta Diversity: unweighted UniFrac distances Visualization

In [89]:
unweighted_unifrac_expermint_title_significance = Visualization.load('core-metrics-results/unweighted-unifrac-expermint-title-significance.qzv')

#### Alpha group significance Visualization
 

In [90]:
observed_features_group_significance = Visualization.load('observed_features_group_significance.qzv')

In [91]:
shannon_group_significance = Visualization.load('shannon_group_significance.qzv')

In [92]:
pielou_e_group_significance = Visualization.load('pielou_e_group_significance.qzv')

In [93]:
faith_pd_group_significance = Visualization.load('faith_pd_group_significance.qzv')

#### Alpha rarefaction plotting Visualization

In [94]:
alpha_rarefaction = Visualization.load('alpha-rarefaction.qzv')

#### PCoA EMPeror Visualization
 

In [95]:
jaccard_emperor = Visualization.load('jaccard_emperor.qzv')

In [96]:
braycurtis_emperor = Visualization.load('braycurtis_emperor.qzv')

In [97]:
unweighted_unifrac_emperor = Visualization.load('unweighted_unifrac_emperor.qzv')

In [98]:
weighted_unifrac_emperor = Visualization.load('weighted_unifrac_emperor.qzv')

#### Beta rarefaction Visualization

In [99]:
unweighted_unifrac_rarefaction = Visualization.load('unweighted_unifrac_rarefaction.qzv')

#### PCoA biplot


In [100]:
unweighted_unifrac_emperor_biplot = Visualization.load('unweighted_unifrac_emperor_biplot.qzv')

#### Mantel test

In [101]:
bc_vs_wu_mantel = Visualization.load('bc_vs_wu_mantel.qzv')

#### Procrustes analysis

In [102]:
bc_vs_wu_procrustes = Visualization.load('bc_vs_wu_procrustes.qzv')